# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Moduł 5: Statystyczne tłumaczenie maszynowe

## Zadanie 1
Zadanie polega na zaimplementowaniu algorytmu Expectation-Maximization w modelu IBM Model 1 do przypasowywania słów. Będzie to fragment modelu, który tłumaczyć będzie z hiszpańskiego na angielski. 

UWAGA: Specjalny token "NULL" pomijamy w implementacji.

Dany jest mini-korpus równoległy angielsko-hiszpański
- "green house" "casa verde"
- "the house" "la casa"
- "the green house" "la casa verde"


In [1]:
import itertools
english = [["green","house"], ["the","house"], ["the", "green", "house"]]
spanish = [["casa", "verde"], ["la", "casa"], ["la", "casa", "verde"]]

W dalszych funkcjach przydatne może być wyznaczenie słownika czyli zbioru słów z korpusu dla danego języka.

In [2]:
import numpy as np

def get_vocabulary(corpus):
    """
    Funkcja zwracająca listę unikalnych słów z korpusu podanego w formacie zmiennej english i spanish
    """
    return set(np.concatenate(corpus))

In [3]:
from nose.tools import assert_set_equal
assert_set_equal(set(get_vocabulary(english)), set(["the", "green", "house"]))

Zainicjalizuj rozkład prawdopodobieństwa tłumaczenia słów rozkładem jednorodnym. Ponieważ zależy nam na prostocie implementacji (a nie efektywności) możemy to prawdopodobieństwo zaimplementować jako zwykły słownik, który będzie przyjmował na wejściu krotkę dwóch słów. Słownik nazwij `translation_prob` z kluczami (słowo_es, słowo_en).

In [4]:
def initalize_translation_prob(corpus1, corpus2):
    pairs = list(itertools.product(get_vocabulary(corpus2), get_vocabulary(corpus1)))
    translation_prob = {p: 1/len(get_vocabulary(corpus2)) for p in pairs}
    return translation_prob
        
translation_prob = initalize_translation_prob(english, spanish)

Wypisz zaincjalizowany słownik, żeby upewnić się że wynik jest prawidłowy.

In [5]:
translation_prob

{('la', 'the'): 0.3333333333333333,
 ('la', 'green'): 0.3333333333333333,
 ('la', 'house'): 0.3333333333333333,
 ('verde', 'the'): 0.3333333333333333,
 ('verde', 'green'): 0.3333333333333333,
 ('verde', 'house'): 0.3333333333333333,
 ('casa', 'the'): 0.3333333333333333,
 ('casa', 'green'): 0.3333333333333333,
 ('casa', 'house'): 0.3333333333333333}

Zaimplementuj pierwszy krok algorytmu EM. Wyznacz wartości oczekiwane zmiennych przypisania słowa we wszystkich zdaniach w korpusie (oznaczane na wykładzie jako `a`).

In [6]:
import pandas as pd

def calculate_expectation(corpora1, corpora2, translation_prob):
    """
    Procedura wykonująca krok "E" algorytmu EM
    Wynikiem powinny być wartości oczekiwane dla zmiennej przypisań słów w zdaniach 
    (reprezentacja dowolna, nie weryfikowana przez sprawdzarkę)
    """
    dfs = []
    for s1, s2 in zip(corpora2, corpora1):
        df = pd.DataFrame(index=set(s1), columns=set(s2))
        for p in translation_prob.keys():
            if p in itertools.product(s1, s2):
                df.loc[p[0], p[1]] = translation_prob[p]
        df = df/df.sum()
        dfs.append(df)
    return dfs

assigment_expected_values = calculate_expectation(english, spanish, translation_prob)

Wypisz wartości oczekiwane zmiennych przypisań, aby zobaczyć jak wyglądają. Powinny one również prezentować całkowity brak wiedzy o przypisaniach (rozkłady jednorodne).

In [7]:
assigment_expected_values

[      green house
 verde   0.5   0.5
 casa    0.5   0.5,       the house
 casa  0.5   0.5
 la    0.5   0.5,             the     green     house
 casa   0.333333  0.333333  0.333333
 verde  0.333333  0.333333  0.333333
 la     0.333333  0.333333  0.333333]

Zaimplementuj drugi krok algorytmu EM. Wyznacz nowe `translation_prob` na podstawie oczekiwanych wartości zmiennych przypisań.

In [8]:
def calculate_maximization(corpora1, corpora2, assigment_expected_values):
    wrd_sum = {}
    pairs = {}
    for df in assigment_expected_values:
        for wrd1 in list(df.index):
            for wrd2 in df.columns:
                val = df.loc[wrd1, wrd2]
                pairs[(wrd1, wrd2)] = pairs[(wrd1, wrd2)] + val if pairs.get((wrd1, wrd2))!=None else val
                wrd_sum[wrd2] = wrd_sum[wrd2] + val if wrd_sum.get(wrd2)!=None else val
    for p in pairs.keys():
        pairs[p] = pairs[p]/wrd_sum[p[1]]
    return pairs

translation_prob = calculate_maximization(english, spanish, assigment_expected_values)

In [9]:
from nose.tools import assert_almost_equal
assert_almost_equal(translation_prob[('casa', 'house')], 4/9.)
assert_almost_equal(translation_prob[('la', 'house')], 5/18.)

Wywołaj w pętli 10 kroków algorytmu EM i zaobserwuj jak zmieniają się prawdopodobieństwa dla tłumacznienia "house".

In [10]:
for i in range(10):
    assigment_expected_values = calculate_expectation(english, spanish, translation_prob)
    translation_prob = calculate_maximization(english, spanish, assigment_expected_values)
    print([(i,j) for i,j in translation_prob.items() if i[1] == "house"])
    print("---")


[(('verde', 'house'), 0.22079772079772084), (('casa', 'house'), 0.5584045584045584), (('la', 'house'), 0.22079772079772084)]
---
[(('verde', 'house'), 0.16805384111904528), (('casa', 'house'), 0.6638923177619094), (('la', 'house'), 0.16805384111904528)]
---
[(('verde', 'house'), 0.12335156766127482), (('casa', 'house'), 0.7532968646774505), (('la', 'house'), 0.12335156766127482)]
---
[(('verde', 'house'), 0.08801990153205524), (('casa', 'house'), 0.8239601969358894), (('la', 'house'), 0.08801990153205524)]
---
[(('verde', 'house'), 0.061511685865775545), (('casa', 'house'), 0.8769766282684488), (('la', 'house'), 0.061511685865775545)]
---
[(('verde', 'house'), 0.042351684951809056), (('casa', 'house'), 0.9152966300963818), (('la', 'house'), 0.042351684951809056)]
---
[(('verde', 'house'), 0.02885878646072998), (('casa', 'house'), 0.94228242707854), (('la', 'house'), 0.02885878646072998)]
---
[(('verde', 'house'), 0.019525050381416956), (('casa', 'house'), 0.960949899237166), (('la', 'h

Wywołaj algorytm EM na poniższym korpusie.

In [11]:
english2 = [["the","dog"], ["the","house"], ["the", "green", "house"]]
polish = [["pies"], ["dom"], ["zielony", "dom"]]

In [12]:
def translate(corp1, corp2, n=10):
    translation_prob = initalize_translation_prob(corp1, corp2)
    for i in range(n):
        assigment_expected_values = calculate_expectation(corp1, corp2, translation_prob)
        translation_prob = calculate_maximization(corp1, corp2, assigment_expected_values)
    return translation_prob

Sprawdź jak wyglądają prawdopodobieństwa tłumaczeń po 10 iteracjach.

In [13]:
translate(english2, polish, n=10)

{('pies', 'the'): 0.3333333333333333,
 ('pies', 'dog'): 1.0,
 ('dom', 'the'): 0.6663411458333334,
 ('dom', 'house'): 0.99951171875,
 ('dom', 'green'): 0.5,
 ('zielony', 'the'): 0.0003255208333333333,
 ('zielony', 'green'): 0.5,
 ('zielony', 'house'): 0.00048828125}

Sprawdź czy gdybyś dodał słówko `NULL` to algorytm nauczyłby się wiązać słówko `NULL` na `the`, które nie występuje w języku polskim?

In [14]:
english2 = [["the","dog"], ["the","house"], ["the", "green", "house"]]
polish_null = [["null", "pies"], ["null", "dom"], ["null", "zielony", "dom"]]
translation_prob = translate(english2, polish_null, n=10)

In [15]:
translation_prob

{('pies', 'the'): 3.5095620537396894e-05,
 ('pies', 'dog'): 0.5,
 ('null', 'the'): 0.9884935162981091,
 ('null', 'dog'): 0.5,
 ('dom', 'the'): 0.01146338484277804,
 ('dom', 'house'): 0.49983723958333337,
 ('null', 'house'): 0.49983723958333337,
 ('zielony', 'the'): 8.00323857550188e-06,
 ('zielony', 'green'): 0.3333333333333333,
 ('zielony', 'house'): 0.00032552083333333337,
 ('dom', 'green'): 0.3333333333333333,
 ('null', 'green'): 0.3333333333333333}

Jeśli wywołałbyś EM dla pierwszego korpusu równoległego (zmienne `english` i `spanish`) i dołączył tokeny `NULL` to EM tłumaczy NULL jako "casa" i "house" jako "casa" z takimi samymi prawdopodobieństwami. Dlaczego?

Zarówno "casa" jak i "NULL" będą obecne w każdym zdaniu z korpusu, więc nie ma sposobu, by wybrać między tymi tokenami i przypisać im konkretną wartość.